In [3]:
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
df = pd.read_csv('2023.csv')
df = df.rename(columns={'id': 'match_id'})
df

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,MatchReportURL,match_id,Home Score,Away Score,Winner
0,1.0,Fri,2022-08-05,20:00,Crystal Palace,1.2,0–2,1.0,Arsenal,25286.0,Selhurst Park,Anthony Taylor,https://fbref.com/en/matches/e62f6e78/Crystal-...,0,0,2,Away
1,1.0,Sat,2022-08-06,12:30,Fulham,1.2,2–2,1.2,Liverpool,22207.0,Craven Cottage,Andy Madley,https://fbref.com/en/matches/6713c1dc/Fulham-L...,1,2,2,Draw
2,1.0,Sat,2022-08-06,15:00,Tottenham,1.5,4–1,0.5,Southampton,61732.0,Tottenham Hotspur Stadium,Andre Marriner,https://fbref.com/en/matches/09d8a999/Tottenha...,2,4,1,Home
3,1.0,Sat,2022-08-06,15:00,Newcastle Utd,1.7,2–0,0.3,Nott'ham Forest,52245.0,St James' Park,Simon Hooper,https://fbref.com/en/matches/1ac96eb4/Newcastl...,3,2,0,Home
4,1.0,Sat,2022-08-06,15:00,Leeds United,0.8,2–1,1.3,Wolves,36347.0,Elland Road,Robert Jones,https://fbref.com/en/matches/82702941/Leeds-Un...,4,2,1,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,38.0,Sun,2023-05-28,16:30,Everton,1.0,1–0,0.5,Bournemouth,39201.0,Goodison Park,Stuart Attwell,https://fbref.com/en/matches/94de848f/Everton-...,375,1,0,Home
376,38.0,Sun,2023-05-28,16:30,Leicester City,1.4,2–1,1.4,West Ham,32183.0,King Power Stadium,Simon Hooper,https://fbref.com/en/matches/a96c9915/Leiceste...,376,2,1,Home
377,38.0,Sun,2023-05-28,16:30,Aston Villa,2.8,2–1,1.4,Brighton,NaN,Villa Park,David Coote,https://fbref.com/en/matches/ac0e65e2/Aston-Vi...,377,2,1,Home
378,38.0,Sun,2023-05-28,16:30,Leeds United,1.5,1–4,2.2,Tottenham,36871.0,Elland Road,Anthony Taylor,https://fbref.com/en/matches/c9c73ddd/Leeds-Un...,378,1,4,Away


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm


# Initialize an empty DataFrame to store all results
agg_df = pd.DataFrame()

# Loop through match IDs; adjust range as needed for actual data processing
for link in tqdm(range(0, 380), desc="Processing Match IDs"):
    url = df.loc[link, 'MatchReportURL']

    # Fetch webpage
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve data for match ID {link}")
        continue
    
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find_all('table')
    num_tables = len(tables)

    headers_list = []
    footer_list = []

    for i, table in enumerate(tables):
        head = table.find('thead')
        footer = table.find('tfoot')

        # Extract headers
        if head:
            headers = [header.text.strip() for header in head.find_all('th') if header.text.strip()]
            headers = [f'H{header}' if i < num_tables // 2 else f'A{header}' for header in headers]

        # Extract footer data
        if footer:
            footer_data = [cell.text.strip() for cell in footer.find_all('td') if cell.text.strip()]
            headers_list += headers[-len(footer_data):]
            footer_list += footer_data

    # Create sub DataFrame for the extracted data
    sub_df = pd.DataFrame([footer_list], columns=headers_list)
    sub_df['match_id'] = df.loc[link, 'match_id']

    # Concatenate to aggregate DataFrame
    agg_df = pd.concat([agg_df, sub_df], ignore_index=True)

    # Delay for a respectful scraping interval
    time.sleep(30)

agg_df


Processing Match IDs: 100%|██████████| 380/380 [3:17:05<00:00, 31.12s/it]  


,HMin,HGls,HAst,HPK,HPKatt,HSh,HSoT,HCrdY,HCrdR,HTouches,...,AInt,ATklW,APKwon,APKcon,AOG,ARecov,AWon,ALost,AWon%,match_id
0,990,0,0,0,0,10,2,1,0,726,...,9,16,0,0,0,55,14,10,58.3,0
1,990,2,1,1,1,8,2,2,0,474,...,10,9,0,1,0,63,13,23,36.1,1
2,990,3,3,0,0,18,8,3,0,709,...,13,6,0,0,1,56,11,13,45.8,2
3,990,2,1,0,0,23,9,0,0,634,...,10,11,0,0,0,52,16,12,57.1,3
4,990,1,1,0,0,12,4,2,0,515,...,14,11,0,0,1,65,7,9,43.8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,990,1,0,0,0,13,6,1,0,469,...,6,7,0,0,0,53,18,21,46.2,375
376,990,2,2,0,0,13,4,1,0,676,...,3,10,0,0,0,53,5,11,31.3,376
377,990,2,2,0,0,12,5,4,0,413,...,3,2,0,0,0,46,11,12,47.8,377
378,990,1,1,0,0,19,2,3,0,636,...,12,12,0,0,0,67,25,30,45.5,378


In [10]:
merged_df = df.merge(agg_df)

In [11]:
from pathlib import Path

filepath = Path('final_2023.csv')  
merged_df.to_csv(filepath, index=False)